# Project 1 - Classification
We will build a classifier that guesses whether a movie is romance or action, using only the numbers of times words appear in the movies's screenplay. We'll build a k-nearest-neighbors classifier and test a classifier on data.

In [1]:
import numpy as np
import math
import pandas as pd

# These lines set up the plotting functionality and formatting.
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# 1. The Dataset

In this project, we are exploring movie screenplays. We'll be trying to predict each movie's genre from the text of its screenplay. In particular, we have compiled a list of 5,000 words that might occur in the dialog of a movie. For each movie, our dataset tells us the frequency with which each of these words occurs in its screenplay. All words have been converted to lowercase.

Run the cell below to read the `movies` table. **It may take up to a minute to load.**

In [5]:
movies = pd.read_csv('movies.csv')
movies.head()

,Title,Genre,Year,Rating,# Votes,# Words,i,the,to,a,...,foster,pub,vegetarian,garrison,grammoo,chimney,bikini,richter,psychopath,fling
0,the terminator,action,1984,8.1,183538,1849,0.040022,0.043807,0.025419,0.024878,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,batman,action,1989,7.6,112731,2836,0.051481,0.033850,0.023977,0.028209,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,tomorrow never dies,action,1997,6.4,47198,4215,0.028707,0.054330,0.030368,0.021827,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000237,0.0
3,batman forever,action,1995,5.4,77223,3032,0.036609,0.042216,0.020449,0.031003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,supergirl,action,1984,4.1,6576,3842,0.041905,0.032275,0.028891,0.026288,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


The above cell prints a few columns of the row for the action movie *The Matrix*.  The movie contains 3792 words. The word "it" appears 115 times, as it makes up  $\frac{115}{3792} \approx 0.030327$ of the words in the movie. The word "not" appears 33 times, as it makes up $\frac{33}{3792} \approx 0.00870253$ of the words. The word "fling" doesn't appear at all.

This numerical representation of a body of text, one that describes only the frequencies of individual words, is called a bag-of-words representation. A lot of information is discarded in this representation: the order of the words, the context of each word, who said what, the cast of characters and actors, etc. However, a bag-of-words representation is often used for machine learning applications as a reasonable starting point, because a great deal of information is also retained and expressed in a convenient and compact format. In this project, we will investigate whether this representation is sufficient to build an accurate genre classifier.

All movie titles are unique. The `row_title` function and `column_title` function provides fast access to the one row for each title. 

In [6]:
def row_title(title):
    return movies[movies["Title"]==title]

In [7]:
def column_title(title, column):
    return np.array(movies[movies["Title"]==title][column])[0]

For example, the fastest way to find the frequency of "hey" in the movie *The Terminator* is to access the `'hey'` column element with column_title function.

In [8]:
column_title("the terminator", "hey")

0.000540833

This dataset was extracted from [a Kaggle dataset from Cornell University](https://www.kaggle.com/Cornell-University/movie-dialog-corpus). After transforming the dataset (e.g., converting the words to lowercase, removing the naughty words, and converting the counts to frequencies), we created this new dataset containing the frequency of 5000 common words in each movie.

In [9]:
print('Words with frequencies:', movies.shape[1]-6) 
print('Movies with genres:', movies.shape[0])

Words with frequencies: 5000
Movies with genres: 242


Using value_counts function, we investigate what Genres we are going to try to predict. 

In [12]:
movies['Genre'].value_counts()

action     135
romance    107
Name: Genre, dtype: int64

## 1.2. Splitting the dataset
We're going to use our `movies` dataset for two purposes.

1. First, we want to *train* movie genre classifiers.
2. Second, we want to *test* the performance of our classifiers.

Hence, we need two different datasets: *training* and *test*.

The purpose of a classifier is to classify unseen data that is similar to the training data. Therefore, we must ensure that there are no movies that appear in both sets. We do so by splitting the dataset randomly. The dataset has already been permuted randomly, so it's easy to split.  We just take the top for training and the rest for test. 

We separate the datasets into two tables:

In [13]:
# Here we have defined the proportion of our data
# that we want to designate for training as 17/20ths
# of our total dataset.  3/20ths of the data is
# reserved for testing.

training_proportion = 17/20

num_movies = movies.shape[0]
num_train = int(num_movies * training_proportion)
num_valid = num_movies - num_train

train_movies = movies.take(np.arange(num_train))
test_movies = movies.take(np.arange(num_train, num_movies))

print("Training: ",   train_movies.shape[0], ";",
      "Test: ",       test_movies.shape[0])

Training:  205 ; Test:  37


# 2. K-Nearest Neighbors

K-Nearest Neighbors (k-NN) is a classification algorithm.  Given some *attributes* (also called *features*) of an unseen example, it decides whether that example belongs to one or the other of two categories based on its similarity to previously seen examples. Predicting the category of an example is called *labeling*, and the predicted category is also called a *label*.

## 2.1. Classifying a movie

In k-NN, we classify a movie by finding the `k` movies in the *training set* that are most similar according to the features we choose. We call those movies with similar features the *nearest neighbors*.  The k-NN algorithm assigns the movie to the most common category among its `k` nearest neighbors.

Let's limit ourselves to just 2 features for now, so we can plot each movie.  The features we will use are the proportions of the words "money" and "feel" in the movie.  Taking the movie "Batman Returns" (in the test set), 0.000502 of its words are "money" and 0.004016 are "feel". This movie appears in the test set, so let's imagine that we don't yet know its genre.

First, we need to make our notion of similarity more precise.  We will say that the *distance* between two movies is the straight-line distance between them when we plot their features in a scatter diagram. This distance is called the Euclidean distance, whose formula is $\sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$.

For example, in the movie *Titanic* (in the training set), 0.0009768 of all the words in the movie are "money" and 0.0017094 are "feel".  Its distance from *Batman Returns* on this 2-word feature set is $\sqrt{(0.000502 - 0.0009768)^2 + (0.004016 - 0.0017094)^2} \approx 0.00235496$.  (If we included more or different features, the distance could be different.)

A third movie, *The Avengers* (in the training set), is 0 "money" and 0.001115 "feel".

The function below creates a plot to display the "money" and "feel" features of a test movie and some training movies. As you can see in the result, *Batman Returns* is more similar to *Titanic* than to *The Avengers* based on these features. However, we know that *Batman Returns* and *The Avengers* are both action movies, so intuitively we'd expect them to be more similar. Unfortunately, that isn't always the case. We'll discuss this more later.

We compute the distance between the two action movies, *Batman Returns* and *The Avengers*, using the `money` and `feel` features only.

In [14]:
batman = row_title("batman returns") 
avengers = row_title("the avengers")

x1 = column_title("batman returns", "money")
y1 = column_title("batman returns", "feel")

x2 = column_title("the avengers", "money")
y2 = column_title("the avengers", "feel")


action_distance = ((x1-x2)**2 + (y1-y2)**2)**0.5

action_distance

0.0029437356216700243

Let's make the function `distance_two_features` that computes the Euclidean distance between any two movies, using two features. The last two lines call your function to show that *Batman Returns* is closer to *The Terminator* than *The Avengers*. 

In [16]:
def distance_two_features(title0, title1, x_feature, y_feature):
    """Compute the distance between two movies with titles title0 and title1
    
    Only the features named x_feature and y_feature are used when computing the distance.
    """
    
    x1 = column_title(title0, x_feature)
    y1 = column_title(title0, y_feature)

    x2 = column_title(title1, x_feature)
    y2 = column_title(title1, y_feature)
    
    action_distance = ((x1-x2)**2 + (y1-y2)**2)**0.5
    
    return action_distance

for movie in np.array(["the terminator", "the avengers"]):
    movie_distance = distance_two_features(movie, "batman returns", "money", "feel")
    print(movie, 'distance:\t', movie_distance)

the terminator distance:	 0.0018531387547749904
the avengers distance:	 0.0029437356216700243


We define the function `distance_from_batman_returns` so that it works as described in its documentation. 

In [17]:
def distance_from_batman_returns(title):
    """The distance between the given movie and "batman returns", based on the features "money" and "feel".
    
    This function takes a single argument:
      title: A string, the name of a movie.
    """
    
    return distance_two_features("batman returns", title, "money", "feel")

distance_from_batman_returns("the terminator")

0.0018531387547749904

Using the features `"money" and "feel"`, what are the names and genres of the 7 movies in the **training set** closest to "batman returns"?  To answer this question, we make a table named `close_movies` containing those 7 movies with columns `"Title"`, `"Genre"`, `"money"`, and `"feel"`, as well as a column called `"distance"` that contains the distance from "batman returns".  The table should be **sorted in ascending order by `distance`**.

In [18]:
a = train_movies.shape[0]
col = []
for i in range(a):
    d0 = distance_from_batman_returns(train_movies['Title'][i])
    col.append(d0)
    
sorted_movies=train_movies
sorted_movies = sorted_movies.assign(diff_col=pd.Series(col))
sorted_movies = sorted_movies.loc[:, sorted_movies.columns.isin(["Title", "Genre", "diff_col"])]
close_movies = sorted_movies.sort_values("diff_col")[0:7]
close_movies

,Title,Genre,diff_col
62,the bridges of madison county,romance,0.000323
204,the fisher king,romance,0.000525
82,broadcast news,romance,0.000590
25,hellboy,action,0.000834
122,as good as it gets,romance,0.000878
198,spider-man,action,0.000903
113,harold and maude,romance,0.001112


Next, we'll clasify "batman returns" based on the genres of the closest movies. 

To do so, we define the function `most_common` so that it works as described in its documentation below.

In [19]:
def most_common(label, table):
    return table[label].value_counts().reset_index()["index"][0]
most_common('Genre', close_movies)

'romance'

# 3. Features

Now, we're going to extend our classifier to consider more than two features at a time.

Euclidean distance still makes sense with more than two features. For `n` different features, we compute the difference between corresponding feature values for two movies, square each of the `n`  differences, sum up the resulting numbers, and take the square root of the sum.

We write a function to compute the Euclidean distance between two **arrays** of features of *arbitrary* (but equal) length and use it to compute the distance between the first movie in the training set and the first movie in the test set, *using all of the features*.


In [20]:
def distance(features1, features2):
    """The Euclidean distance between two arrays of feature values."""
    return (np.sum(features1-features2)**2)**0.5

distance_first_to_first = distance(np.array(train_movies.iloc[0,6:]), np.array(test_movies.iloc[0,6:]))
distance_first_to_first

1.3399999987384874e-07

We will get started on selecting more effective features for distinguishing romance from action movies. The plot below (generated from data) shows the average number of times each word occurs in a romance movie on the horizontal axis and the average number of times it occurs in an action movie on the vertical axis. 

![alt text](word_plot.png "Title")

Using the plot above, we choose 20 common words that we think might let us distinguish between romance and action movies.

In [23]:
# Set my_20_features to an array of 20 features (strings that are column labels)

my_20_features = ["power","run","marri","captain","move","home","system","weve","ship","hello","three","letter","write","secur"]

train_20 = train_movies.loc[:,train_movies.columns.isin(my_20_features)]
test_20 = test_movies.loc[:, test_movies.columns.isin(my_20_features)]
train_20.head()

,run,home,three,move,weve,marri,hello,power,captain,ship,write,system,letter,secur
0,0.001082,0.001082,0.000000,0.001082,0.000000,0.000000,0.000000,0.000541,0.000000,0.00000,0.000000,0.000541,0.000000,0.000000
1,0.000705,0.000353,0.000000,0.001058,0.000705,0.000000,0.000000,0.000705,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.001186,0.000000,0.000474,0.000237,0.000712,0.000712,0.000474,0.000237,0.001186,0.00261,0.000474,0.000000,0.000000,0.000237
3,0.000989,0.000660,0.000660,0.000330,0.000330,0.000000,0.000000,0.000330,0.000000,0.00000,0.000330,0.000000,0.001649,0.000000
4,0.000521,0.001041,0.000260,0.001041,0.000000,0.000260,0.000260,0.004164,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


Next, let's classify the first movie from our test set using these features.

In [24]:
test_movies.head(1)

,Title,Genre,Year,Rating,# Votes,# Words,i,the,to,a,...,foster,pub,vegetarian,garrison,grammoo,chimney,bikini,richter,psychopath,fling
205,the mummy,action,1999,6.9,96736,3115,0.035313,0.047833,0.032424,0.020225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


As before, we want to look for the movies in the training set that are most alike our test movie.  We will calculate the Euclidean distances from the test movie (using the 20 selected features) to all movies in the training set.

In [26]:
def fast_distances(test_row, train_rows):
    """An array of the distances between test_row and each row in train_rows.

    Takes 2 arguments:
      test_row: A row of a table containing features of one
        test movie (e.g., test_20.row(0)).
      train_rows: A table of features (for example, the whole
        table train_20)."""
    assert train_rows.shape[1] < 50, "Make sure you're not using all the features of the movies table."
    counts_matrix = train_rows.as_matrix(columns=train_rows.columns.values)
    diff = np.tile(np.array(test_row), [counts_matrix.shape[0], 1])- counts_matrix
    np.random.seed(0) # For tie breaking purposes
    distances = np.squeeze(np.asarray(np.sqrt(np.square(diff).sum(1))))
    eps = np.random.uniform(size=distances.shape)*1e-10 #Noise for tie break
    distances = distances + eps
    return distances

We use the `fast_distances` function provided above to compute the distance from the first movie in the test set to all the movies in the training set, **using your set of 20 features**. We make a new table called `genre_and_distances` with one row for each movie in the training set and two columns:
* The `"Genre"` of the training movie
* The `"Distance"` from the first movie in the test set 

`genre_and_distances` is **sorted in increasing order by distance to the first test movie**.

In [27]:
genre_and_distances = pd.DataFrame(data={"Genre": train_movies["Genre"],
                                         "Distances":  fast_distances(test_20.iloc[0,:], train_20)})
genre_and_distances.sort_values("Distances").head()

,Genre,Distances
12,action,0.000558
71,romance,0.000849
149,action,0.000863
39,romance,0.000900
50,romance,0.000965


We compute the 5-nearest neighbors classification of the first movie in the test set.  That is, decide on its genre by finding the most common genre among its 5 nearest neighbors, according to the distances we've calculated.  Then we check whether your classifier chose the right genre.  (Depending on the features we chose, our classifier might not get this movie right, and that's okay.)

In [30]:
genre = most_common("Genre",genre_and_distances.sort_values("Distances")[0:5])
print(genre)
print("yes" if genre==test_movies.iloc[0,:]['Genre'] else "no")

romance
no


## 3.2. A classifier function

Now we can write a single function that encapsulates the whole process of classification.

We write a function called `classify`.  It takes the following four arguments:
* A row of features for a movie to classify.
* A table with a column for each feature.
* An array of classes that has as many items as the previous table has rows, and in the same order.
* `k`, the number of neighbors to use in classification.

It returns the class a `k`-nearest neighbor classifier picks for the given row of features (the string `'Romance'` or the string `'Action'`).

In [31]:
def classify(test_row, train_rows, train_classes, k):
    distances=fast_distances(test_row,train_rows)
    genre_and_distances = pd.DataFrame(data={"Class": train_classes, "Distances": distances})
    sor_ted=genre_and_distances.sort_values(by="Distances")[0:k]
    most=most_common("Class",sor_ted)
    return most
classify(test_20.iloc[20,:],train_20,train_movies["Genre"],9)

'romance'

Finally, when we evaluate our classifier, it will be useful to have a classification function that is specialized to use a fixed training set and a fixed value of `k`.

We create a classification function that takes as its argument a row containing our 20 features and classifies that row using the 11-nearest neighbors algorithm with `train_20` as its training set.

In [33]:
def classify_one_argument(row):
    return classify(row,train_20,train_movies["Genre"],11)

classify_one_argument(test_20.iloc[0,:])

'romance'

## 3.3. Evaluating classifier

Now that it's easy to use the classifier, let's see how accurate it is on the whole test set.

In [34]:
test_guesses = test_20.apply(classify_one_argument, axis=1)
test_guesses
proportion_correct = np.count_nonzero(test_guesses== test_movies["Genre"])/test_movies.shape[0]*100
proportion_correct

89.1891891891892

Wohooo 89%!!!